<a href="https://colab.research.google.com/github/BrMrtn/GoogleColab/blob/main/K%C3%A9pfeldolgoz%C3%A1s/khf1_3_Tatcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Könyvtárak importálása

In [ ]:
import cv2
import numpy as np

képek letölétése

In [ ]:
!wget https://raw.githubusercontent.com/BrMrtn/GoogleColab/main/resources/Lawrence.png
!wget https://raw.githubusercontent.com/BrMrtn/GoogleColab/main/resources/Reynolds.png

In [ ]:
# Load Haar cascade classifiers for face, eyes, and mouth
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')  # Smile as a proxy for mouth

In [ ]:
def create_thatcher_illusion(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect face
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        # Draw rectangle around the face (optional)
        # cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Region of interest for eyes and mouth within the face
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]

        # Detect eyes
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 10)
        eye_regions = []
        for (ex, ey, ew, eh) in eyes[:2]:  # Limit to first two detected eyes
            eye_region = roi_color[ey:ey+eh, ex:ex+ew]
            eye_regions.append((ex, ey, ew, eh, eye_region))

        # Rotate each eye region
        for (ex, ey, ew, eh, eye_region) in eye_regions:
            rotated_eye = cv2.rotate(eye_region, cv2.ROTATE_180)
            roi_color[ey:ey+eh, ex:ex+ew] = rotated_eye

        # Detect mouth (using smile classifier as a proxy)
        mouths = mouth_cascade.detectMultiScale(roi_gray, 1.7, 22)
        if len(mouths) > 0:
            (mx, my, mw, mh) = mouths[0]  # Take the first detected mouth
            mouth_region = roi_color[my:my+mh, mx:mx+mw]

            # Rotate mouth region
            rotated_mouth = cv2.rotate(mouth_region, cv2.ROTATE_180)
            roi_color[my:my+mh, mx:mx+mw] = rotated_mouth

    # Rotate the entire image by 180 degrees
    thatcher_image = cv2.rotate(image, cv2.ROTATE_180)

    # Save and display the result
    cv2.imwrite('thatcher_illusion.png', thatcher_image)
    cv2.imshow('Thatcher Illusion', thatcher_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
create_thatcher_illusion('/path/to/your/image.jpg')